In [1]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
%matplotlib inline
import re
from tqdm import notebook
import seaborn as sns
import tensorflow as tf
from scipy import stats
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.preprocessing import StandardScaler

In [2]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin'] 
raw_dataset = pd.read_csv('auto-mpg.data', names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [3]:
##drop missingness
dataset.isna().sum()
dataset=dataset.dropna()

### Scramble the training and testing

In [14]:
train=dataset.sample(frac=0.6,random_state=123)
test=dataset.drop(train.index)
dtnew=pd.concat([train,test])


In [15]:
#recode categorical
Ori=pd.get_dummies(dtnew['Origin'],drop_first=True)
Cylinders=pd.get_dummies(dtnew['Cylinders'],drop_first=True)

In [16]:
#standardize the continous variable
continuous=dtnew.drop(['Cylinders','Origin'],axis=1)

In [17]:
scaler = StandardScaler()
scaler.fit(continuous)
continuous=scaler.transform(continuous)

In [20]:
dtnew=np.c_[continuous,Ori,Cylinders]

In [21]:
#put the outcome to last column
dtnew=dtnew[:,::-1]

### Output data

In [23]:
np.save('mpgdt',dtnew)

In [25]:
test.shape

(157, 8)